<a href="https://colab.research.google.com/github/arpitarumma/Hospital-Management--using-SHA-256-E2EE-AES-CBC/blob/main/Medyscrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import sqlite3
import base64
import hashlib
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

# Database Setup
db_path = "ciphergate.db"

def init_db():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Users Table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        security_question TEXT NOT NULL,
        security_answer_hash TEXT NOT NULL
    )
    """)

    # Prescriptions Table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prescriptions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_name TEXT NOT NULL,
        report TEXT NOT NULL,
        appointment_code TEXT NOT NULL,
        doctor_code TEXT NOT NULL
    )
    """)

    # Login Logs Table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS login_logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL,
        login_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)

    conn.commit()
    conn.close()

init_db()

# 🔐 SHA-256 Hashing
def hash_sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()

# 🔐 AES Encryption Functions
def encrypt_data(data, key):
    key = key.ljust(16, '0')[:16].encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode(), AES.block_size))
    return base64.b64encode(cipher.iv + ct_bytes).decode()

def decrypt_data(encrypted_data, key):
    try:
        key = key.ljust(16, '0')[:16].encode()
        raw = base64.b64decode(encrypted_data)
        iv = raw[:16]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(raw[16:]), AES.block_size).decode()
    except Exception:
        return "❌ Decryption failed - Wrong key!"

# 🔑 User Registration
def register_user(username, password, security_question, security_answer):
    if not username or not password or not security_question or not security_answer:
        return "❌ All fields are required!"

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        password_hash = hash_sha256(password)
        security_answer_hash = hash_sha256(security_answer)

        cursor.execute("INSERT INTO users (username, password_hash, security_question, security_answer_hash) VALUES (?, ?, ?, ?)",
                       (username, password_hash, security_question, security_answer_hash))
        conn.commit()
        conn.close()
        return "✅ Registration Successful!"
    except sqlite3.IntegrityError:
        conn.close()
        return "❌ Username already exists!"

# 🔑 User Login
def login_user(username, password):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT password_hash, security_question FROM users WHERE username=?", (username,))
    user = cursor.fetchone()

    if user and hash_sha256(password) == user[0]:
        cursor.execute("INSERT INTO login_logs (username) VALUES (?)", (username,))
        conn.commit()
        conn.close()
        return f"✅ Login successful! Security Question: {user[1]}"

    conn.close()
    return "❌ Incorrect username or password!"

# 📜 Login History Table
def view_login_history():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT username, login_time FROM login_logs ORDER BY login_time DESC")
    logs = cursor.fetchall()
    conn.close()

    if not logs:
        return "No login history found."

    table_html = "<table border='1'><tr><th>Username</th><th>Login Time</th></tr>"
    for log in logs:
        table_html += f"<tr><td>{log[0]}</td><td>{log[1]}</td></tr>"
    table_html += "</table>"

    return table_html

# 🔑 Verify Security Answer
def verify_security_answer(username, security_answer):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT security_answer_hash FROM users WHERE username=?", (username,))
    user = cursor.fetchone()
    conn.close()

    if user and hash_sha256(security_answer) == user[0]:
        return "✅ Authentication successful!"
    return "❌ Incorrect answer!"

# 🏥 Add Encrypted Prescription (With Appointment & Doctor Code)
def add_prescription(patient_name, report, appointment_code, doctor_code):
    access_code = appointment_code + doctor_code
    encrypted_report = encrypt_data(report, access_code)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO prescriptions (patient_name, report, appointment_code, doctor_code) VALUES (?, ?, ?, ?)",
                   (patient_name, encrypted_report, appointment_code, doctor_code))
    conn.commit()
    conn.close()
    return "✅ Prescription added securely!"

# 📜 View Decrypted Prescription
def view_prescription(patient_name, appointment_code, doctor_code):
    access_code = appointment_code + doctor_code

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT report FROM prescriptions WHERE patient_name=?", (patient_name,))
    prescription = cursor.fetchone()
    conn.close()

    if prescription:
        decrypted_text = decrypt_data(prescription[0], access_code)
        return f"📝 {patient_name}'s Prescription: {decrypted_text}"

    return "❌ No matching prescription found!"

# 🎭 Gradio UI
with gr.Blocks() as ciphergate_ui:
    gr.Markdown("# 👩‍⚕️🩺🏥 Welcome to **MEDISCRYPT-Securing healthcare, One HASH at a time**")

    with gr.Tab("👤 Register"):
        reg_username = gr.Textbox(label="Username")
        reg_password = gr.Textbox(label="Password", type="password")
        security_question = gr.Textbox(label="Security Question")
        security_answer = gr.Textbox(label="Answer", type="password")
        register_btn = gr.Button("Register")
        reg_status = gr.Textbox()
        register_btn.click(register_user, inputs=[reg_username, reg_password, security_question, security_answer], outputs=reg_status)

    with gr.Tab("🔑 Login"):
        login_username = gr.Textbox(label="Username")
        login_password = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        login_status = gr.Textbox()
        login_btn.click(login_user, inputs=[login_username, login_password], outputs=login_status)

    with gr.Tab("📜 Upload Medical Prescription"):
        patient_name = gr.Textbox(label="Patient Name")
        report_text = gr.Textbox(label="Report Content")
        appointment_code = gr.Textbox(label="Patient Appointment Code")
        doctor_code = gr.Textbox(label="Doctor's Access Code")
        add_report_btn = gr.Button("Add Prescription")
        add_report_status = gr.Textbox()
        add_report_btn.click(add_prescription, inputs=[patient_name, report_text, appointment_code, doctor_code], outputs=add_report_status)

    with gr.Tab("🔍 View Prescription"):
        patient_name_search = gr.Textbox(label="Patient Name")
        appointment_code_search = gr.Textbox(label="Appointment Code")
        doctor_code_search = gr.Textbox(label="Doctor Code")
        view_report_btn = gr.Button("View Prescription")
        report_display = gr.Textbox()
        view_report_btn.click(view_prescription, inputs=[patient_name_search, appointment_code_search, doctor_code_search], outputs=report_display)

    with gr.Tab("📜 Login History"):
        view_logs_btn = gr.Button("View Login History")
        logs_display = gr.HTML()
        view_logs_btn.click(view_login_history, outputs=logs_display)

ciphergate_ui.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bfe5d2dcd9a035b442.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
SELECT * FROM users;